In [26]:
import requests
from bs4 import BeautifulSoup
import re
import pygsheets
import pandas as pd

keypath = 'C:\\Users\Kaspe\Documents\AuthKeys\superfoods-383120-9880791a7067.json'
gc = pygsheets.authorize(service_file=keypath)
sh = gc.open("Super Foods")

In [27]:
def CleanPrice(numElement):
    numElement = numElement.get_text().strip().replace(u'\xa0', u' ')
    numElement = numElement.replace(u',', u'.')
    x = re.findall("\d+\.\d+", numElement)
    if(x):
        numElement = re.findall("\d+\.\d+", numElement)[0]
        return numElement
    else:
        return 0

def InsertIcaProduct(df, url):
    response = requests.get(url)
    soup_data = BeautifulSoup(response.text, 'html.parser')
   
    title  = soup_data.findAll('h1', {"class": "heading__Base-sc-1vuwqc7-0-h1 sc-ehCIER ZBWKl rZSBo"})
    title = title[0].text.strip()

    kg_price = soup_data.findAll('span', {"class": "text__Text-sc-6l1yjp-0 bhymDA"})
    if(kg_price):
        kg_price = CleanPrice(kg_price[1])
    
    price = soup_data.findAll('div', {"class" : "spacing__Spacing-sc-ngu0v9-0 lbarHM"})
    if(price):
        price = CleanPrice(price[0])
    
    contents = soup_data.findAll('div', {"class": "static-content-wrapper__StaticContentWrapper-sc-3z5iao-0 fdAbME"})
    contents.reverse()

    content_ls = contents[0].text.strip().split(",")
    if('kcal' not in contents[0].text):
        content_ls.insert(0, '0')
    if('kJ' not in contents[0].text):
        content_ls.insert(1, '0')
    if('Fett' not in contents[0].text):
        content_ls.insert(2, '0')
    if('mättat' not in contents[0].text):
        content_ls.insert(3, '0')
    if('Kolhydrat' not in contents[0].text):
        content_ls.insert(4, '0')
    if('socker' not in contents[0].text):
        content_ls.insert(5, '0')
    if('Fiber' not in contents[0].text):
        content_ls.insert(6, '0')
    if('Protein' not in contents[0].text):
        content_ls.insert(7, '0')
    if('Salt' not in contents[0].text):
        content_ls.insert(8, '0')

    for idx in range(len(content_ls)):
        if(content_ls[idx]):
            x = re.findall(r'\d+(?:\.\d+)?', content_ls[idx])
            if(x):
                content_ls[idx] = x[0]

    df.loc[len(df)] = [title, content_ls[0],content_ls[1],content_ls[2],content_ls[3],content_ls[4],content_ls[5],content_ls[6],content_ls[7],content_ls[8], kg_price, price, url]
    return [df]

In [28]:
df = pd.DataFrame(columns=['product','kcal', 'kj', 'fat_g','sat_fat_g', 'carb_g', 'sugar_g', 'fiber_g','protein_g','salt_g', 'kg_price_kr', 'price_kr','link'])

for value in sh[1].get_all_values():
    if(value[0]!=''):
        InsertIcaProduct(df, value[0])

df.head()

IndexError: list index out of range

In [ ]:
# Export to google sheet and local csv 
df.to_csv('scraped_foods.csv')

wks = sh[0]
wks.set_dataframe(df,(1,1))